# Creating supercells with pymatgen

The [Pymatgen](https://pymatgen.org/) python library allows to setup
solid-state calculations using a flexible set of classes as well as an API
to an online data base of structures. Its `Structure` and `Lattice`
objects are directly supported by the DFTK `load_atoms` and `load_lattice`
functions, such that DFTK may be readily used to run calculation on systems
defined in pymatgen. Using the `pymatgen_structure` function a conversion
from DFTK to pymatgen structures is also possible. In the following we
use this to create a silicon supercell and find its LDA ground state
using direct minimisation.

First we setup the silicon lattice in DFTK.

In [1]:
using DFTK

a = 10.263141334305942  # Lattice constant in Bohr
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms = [Si => [ones(3)/8, -ones(3)/8]];

Next we make a `[2, 2, 2]` supercell using pymatgen

In [2]:
pystruct = pymatgen_structure(lattice, atoms)
pystruct.make_supercell([2, 2, 2])
lattice = load_lattice(pystruct)
atoms = [Si => [s.frac_coords for s in pystruct.sites]];

Setup an LDA model and discretize using
a single kpoint and a small `Ecut` of 5 Hartree.

In [3]:
model = model_LDA(lattice, atoms)
basis = PlaneWaveBasis(model, 5, kgrid=(1, 1, 1))

PlaneWaveBasis (Ecut=5.0, 1 kpoints)

Find the ground state using direct minimisation (always using SCF is boring ...)

In [4]:
scfres = direct_minimization(basis, tol=1e-5);

Iter     Function value   Gradient norm 
     0     1.125472e+02     1.507297e+00
 * time: 0.07879996299743652
     1     1.098487e+01     9.236241e-01
 * time: 1.3282480239868164
     2    -1.180331e+01     9.461582e-01
 * time: 1.434692144393921
     3    -3.426997e+01     6.973646e-01
 * time: 1.5752241611480713
     4    -4.789223e+01     5.054297e-01
 * time: 1.743704080581665
     5    -5.722717e+01     2.067940e-01
 * time: 1.8743000030517578
     6    -6.005966e+01     1.052856e-01
 * time: 1.981856107711792
     7    -6.101152e+01     5.178823e-02
 * time: 2.095980167388916
     8    -6.135731e+01     8.161411e-02
 * time: 2.2162270545959473
     9    -6.165202e+01     3.400898e-02
 * time: 2.3169350624084473
    10    -6.183930e+01     2.814700e-02
 * time: 2.4164280891418457
    11    -6.198290e+01     2.231472e-02
 * time: 2.5187020301818848
    12    -6.204794e+01     1.791115e-02
 * time: 2.6256470680236816
    13    -6.210796e+01     1.445340e-02
 * time: 2.7437620162963

In [5]:
scfres.energies

Energy breakdown:
    Kinetic             25.7671061
    AtomicLocal         -18.8557681
    AtomicNonlocal      14.8522655
    Ewald               -67.1831486
    PspCorrection       -2.3569765
    Hartree             4.8485372 
    Xc                  -19.3336820

    total               -62.261666460121
